In [1]:
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import jieba.posseg as pseg
import pickle
import os
import tensorflow.keras as keras
from tensorflow.keras import Sequential , Model
import tensorflow as tf

In [3]:
data = pd.read_csv('main_surname.csv' , header = None).fillna("")
data['内容'] = data[1] + data[6]
data['内容'] = data['内容'].str.replace(r'[\d|a-z|A-Z]' , "" , regex = True)
data.head()

,0,1,2,3,4,5,6,7,内容
0,丁,丁姓，中华姓氏之一，是一个典型的多民族、多源流姓氏，始祖为丁公伋。\n丁姓在宋版《百家姓》中...,东汉\n丁兰：“丁兰刻木事亲”为二十四孝故事之一。\n南宋\n丁大全：1191-1263 南...,,丁姓的支源众多，但大致上，山东为其最早发源地，后在当地发展成为丁姓最大的济阳郡望，山东丁姓亦...,源出一\n第一支见诸于《姓氏考考略》。周武王伐纣之时﹐就有了丁姓的诸侯。但这位丁姓诸侯的详细...,郡望\n丁姓的郡望主要有济阳、济阴、陈郡、谯国、扶风、西河、洛阳等。\n济阳郡：战国时为魏邑...,130,丁姓，中华姓氏之一，是一个典型的多民族、多源流姓氏，始祖为丁公伋。\n丁姓在宋版《百家姓》中...
1,万,万姓的得姓始祖为芮伯万。周初，成王封同姓族人姬良夫于芮（今山西省芮城和陕西大荔一带），遂建芮...,战国\n万章：战国时期邹国齐国人，战国时期孟子门下的得意弟子，是最早扬名于历史的万姓先人。关...,万氏是一个多民族、多源流的古老姓氏，在当今姓氏排行榜上名列第八十八位，属于大姓系列，人口约二...,,源流一\n源于姬姓，出自周王朝芮伯的后裔芮伯万，属于以先祖名字为氏。据史籍《通志•氏族略》记...,郡望\n扶风郡：周朝时期置郡，其时辖地在今陕西省兴平县、咸阳市一带地区。汉朝武帝太初元年(丁...,0,万姓的得姓始祖为芮伯万。周初，成王封同姓族人姬良夫于芮（今山西省芮城和陕西大荔一带），遂建芮...
2,上官,[又，姓氏，读音作yòu(ㄧㄡˋ)]\n又，殷商时期氏族，出自殷墟出土甲骨文《卜辞》中，以氏...,明\n又尚珍：陇西宣德中任南京豹韬卫经历。,,云南昌宁，湖南双峰、新化，江苏滨海，北京等地均有此姓。,,,0,[又，姓氏，读音作ò(ㄧㄡˋ)]\n又，殷商时期氏族，出自殷墟出土甲骨文《卜辞》中，以氏族名...
3,东,粤稽洪荒甫辟，生民浑噩，姓氏未著。自太昊嗣兴帝，号伏羲，以木德王其姓风氏。民之有姓，实自此始...,元\n东良会：？-1357 元朝巩昌人，个性耿直，事亲孝顺，教子爱国。任商州总兵时，遇红巾军...,东氏人口总数在中国的大陆和台湾省均未列入百家姓前三百位，在宋版《百家姓》中排序为第三百六十位...,,粤稽洪荒甫辟，生民浑噩，姓氏未著。自太昊嗣兴帝，号伏羲，以木德王其姓风氏。民之有姓，实自此始...,郡望\n太原郡,0,粤稽洪荒甫辟，生民浑噩，姓氏未著。自太昊嗣兴帝，号伏羲，以木德王其姓风氏。民之有姓，实自此始...
4,严,严姓，源于庄姓，是中国南方典型的姓氏之一。史载，东汉时，因避讳明帝刘庄，改姓为严。奉严忌为始...,战国\n严君疾：？―前300年 是战国中期秦国宗室、将领，秦孝公庶子，秦惠文王异母弟，其母为...,严姓主要由庄姓改姓而来，故一开始就分布广泛。史载，先秦时期，严姓氏族势小力弱，主要集居在西北...,宋朝时期，严姓大约有12万多人，大约占中国人口的o.16%，排在宋朝时的第九十九位。湖南为严...,源于芈姓\n出自楚庄王之孙，以谥为姓。魏晋时严氏有恢复庄氏的，于是形成庄、严两家。两姓原出是...,郡望\n严姓望出天水郡、冯翊郡、华阴郡。\n堂号\n严姓主要堂号有：“天水堂”、“富春堂”、...,0,严姓，源于庄姓，是中国南方典型的姓氏之一。史载，东汉时，因避讳明帝刘庄，改姓为严。奉严忌为始...


In [70]:
def get_tf_idf_array(words): #先返回word矩阵 再返回对应的权值矩阵，如果该第i行没有第j个词，那么weight[i][j] = 0
    tf_vector = CountVectorizer()#token_pattern = '(?u)\\b\\w+\\b')
    cnt_tf = tf_vector.fit_transform(words)
    tf = TfidfTransformer()
    tf_idf =  tf.fit_transform(cnt_tf);
    word = tf_vector.get_feature_names()#获取词袋模型中的所有词语
    weight = tf_idf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
    return word , weight

def get_import_by_sentence(word , weight , sentence , index  , num = 10 , allows = {'a' , 'd' , 'v' ,'x' ,'m' } , name = ""):
    words = []
    #print(sentence[index])
    for i in range(len(word)):
        if weight[index][i] == 0:
            continue;
        word_tmp = ""
        tag = ""
        for w , t in pseg.cut(word[i]):
            word_tmp += w;
            tag += t
        #print(word_tmp , tag)
        if tag in allows:
                words.append(( weight[index][i] , word_tmp))
                print(word_tmp , tag)
                break;
        
    words = sorted(words , key = lambda x : x[0] , reverse = True)[: num]
    words = [i[1] for i in words]
    try:
        words = sorted(words , key = lambda x : sentence[index].index(x))
    except:
        print(words , sentence[index])
    return words

def get_stop_words(path = 'stop_list.txt'  , encoding = 'UTF8'):
    stop_words = [];
    with open(path , encoding = encoding ) as f:
        content_tmp = f.readline();
        while content_tmp:
            stop_words.append(content_tmp.replace('\n' , ""));
            content_tmp = f.readline();
    return stop_words;

def get_words_by_line(name , Data = None ):
    if  str(type(Data)) == "<class \'NoneType\'>":
        Data = data
    Index = {}
    for i in Data[name].dropna().drop_duplicates():
        Index[i] = []
    i = 0
    words = []
    for index , contents in list(zip(list(Data[name])  , list(Data["内容"]))):
        if index != index:
            continue
        now = " ".join(list(filter(lambda x: x not in stop_words , jieba.lcut(contents))));
        if now:
            words.append(now)
            Index[index].append(i);
            i += 1;
        else:
            words.append(now)
    return words , Index

In [71]:
stop_words = get_stop_words()
group_name = 0
sentence , sentence_index = get_words_by_line(group_name)
word , weight = get_tf_idf_array(sentence)
index_num = data[group_name].drop_duplicates()


In [58]:
allows = {'a','v','d','n'}
for j in index_num:
    if j in sentence_index:
        for i in sentence_index[j]:
                print(get_import_by_sentence(word , weight , sentence  ,i  , 1000 ,allows , j))

人口 n
['人口']
一带 n
['一带']
出土 v
['出土']
不算 v
['不算']
一心 d
['一心']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
上于 v
['上于']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一大 a
['一大']
一带 n
['一带']
不愧 a
['不愧']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
世人 n
['世人']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
上党 n
['上党']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
义陵 n
['义陵']
世孙 n
['世孙']
一带 n
['一带']
义陵 n
['义陵']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
儿子 n
['儿子']
一带 n
['一带']
一带 n
['一带']
世居 n
['世居']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
三相 n
['三相']
世德 n
['世德']
不染 v
['不染']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
为辅 v
['为辅']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
世孙 n
['世孙']
人口 n
['人口']
一带 n
['一带']
人口 n
['人口']
临济 n
['临济']
一带 n
['一带']
一带 n
['一带']
亲堂 n
['亲堂']
一带 n
['一带']
一同 d
['一同']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n
['一带']
一带 n

In [62]:
tmp = map(lambda x : list(map(lambda k : {"name" : word[k] , 'symbolSize' : weight[x[0]][k]} , x[1])) , enumerate(weight.argsort(axis = 1)[: , :-51:-1]))
for index , content in zip(index_num , tmp):
    data[index] = content
with open('firstname/static/firstname/model/word', 'wb') as f:
    pickle.dump(data,f)

[['陈郡',
  '家里',
  '经学家',
  '清时',
  '很穷',
  '松柏',
  '土筑',
  '守护',
  '丁宝桢',
  '丁汝昌',
  '丁公',
  '白鹿',
  '丁茂',
  '丁云鹏',
  '拒不受',
  '驯鹿',
  '最孝',
  '自背',
  '魏邑',
  '丁日昌',
  '丁鸿',
  '丁恭',
  '母亲',
  '济阳',
  '丁姓'],
 ['出芮伯万',
  '穿着',
  '周襄王',
  '唐至德',
  '平安',
  '不降',
  '公元前',
  '其时',
  '地区',
  '丁酉',
  '兴平',
  '洛阳',
  '咸阳市',
  '兴平县',
  '一带',
  '万姓',
  '陕西',
  '芮国',
  '芮城',
  '安史之乱',
  '陕西省',
  '时期',
  '扶风',
  '河南',
  '西堂'],
 ['太原市',
  '太史',
  '太史令',
  '太后',
  '太和',
  '太和县',
  '太嗥',
  '太夫人',
  '太子',
  '太学',
  '太守',
  '太宗',
  '太原人',
  '大部分',
  '姓氏',
  '时期',
  '出自',
  '名为',
  '读音',
  '殷商',
  '出土',
  '殷墟',
  '氏族',
  '甲骨文',
  '卜辞'],
 ['命掌',
  '东阿县',
  '风氏',
  '以木德王',
  '民之有',
  '黑龙江省',
  '齐齐哈尔市',
  '阙东',
  '巴东县',
  '大庆市',
  '鹿泉',
  '佳木斯市',
  '营口市',
  '訾者',
  '永登县',
  '多伦多市',
  '城阳区',
  '生民',
  '帝舜七友',
  '武清区',
  '自太昊嗣兴帝',
  '济南市',
  '青阳',
  '太原',
  '东方'],
 ['五堂',
  '文有',
  '四录堂',
  '严忌',
  '严颜',
  '育德堂',
  '最新版',
  '闭门',
  '古秋堂',
  '宜雅堂',
  '为严',
  '帝刘庄',
  '武略',
  '严武',
  '海云堂',
  '武有',
  '著于',
  '总

In [3]:
def MATCH(text1 , text2 , point = 0):
    length =  0
    length1 = len(text1)
    length2 = len(text2)
    for  index1 in range(length1):
        tmp = 0
        for  index2 in range(length2):
            if index1 + tmp < length1 and text1[index1 + tmp] == text2[index2]:
                tmp += 1
            else:
                length = max(length , tmp)
                tmp = 0
        length = max(length , tmp)
    return length / (length1 if point==0 else length2)
def find(array , text):
    real = ""
    length = 0
    index = None
    for Index ,i in enumerate(array):
        tmp = MATCH(i , text)
        if tmp > length:
            real = i
            length = tmp
            index = Index
    return real , index
dynasty = {
"夏朝":[-2070,-1600],
"商（殷）朝":[-1600,-1046],
"周朝":[-1046,-256],
"春秋战国" : [-770,-221],
"秦朝":[-221,-206],
"汉新朝":[-202,263],
"三国时期":[220,280],
"晋朝":[265,420],
"南北齐西魏陈":[386,589],
"隋朝":[581,618],
"唐朝":[618,907],
"五代十国后梁吴越":[907,960],
"宋，金,辽":[960,1279],
"元":[1271,1368],
"明":[1368,1644],
"清朝":[1636,1912],
"中华民国" : [1912,None],
"中华人民共和国":[1945,None]
}

dynasty = list(map( lambda x : x.split('朝')[0], list(dynasty.keys())))
Dynasty = {
"夏朝":[-2070,-1600],          
"商（殷）朝":[-1600,-1046],   
"周朝":[-1046,-256],           
"春秋战国" : [-770,-221],     
"秦朝":[-221,-206],          
"汉朝":[-202,263],            
"三国时期":[220,280],         
"晋朝":[265,420],            
"南北朝":[386,589],          
"隋朝":[581,618],            
"唐朝":[618,907],            
"五代十国":[907,960],         
"宋朝":[960,1279],           
"元朝":[1271,1368],          
"明朝":[1368,1644],          
"清朝":[1636,1912],          
"中华民国" : [1912,None],    
"中华人民共和国":[1945,None]  
}

dynasty_tmp = list(Dynasty.keys())
dynasty


['夏',
 '商（殷）',
 '周',
 '春秋战国',
 '秦',
 '汉新',
 '三国时期',
 '晋',
 '南北齐西魏陈',
 '隋',
 '唐',
 '五代十国后梁吴越',
 '宋，金,辽',
 '元',
 '明',
 '清',
 '中华民国',
 '中华人民共和国']

In [3]:
all = {}
path = "F:/数据库/姓氏/"
for i in os.listdir(path):
    data = ""
    tmp_path = path + i + '/'
    if "历代名人.txt" in os.listdir(tmp_path):
        tmp_path +="历代名人.txt"
    else:
        continue
    with open(tmp_path , 'r' , encoding='utf-8') as f:
        line = f.readline().replace('\n','')
        Type = None
        while line:
            data += line
            if line.replace('国' , '').replace('前', "") == '魏' or line.replace('国' , '').replace('前', "") == '蜀' or line.replace('国' , '').replace('前', "") =='吴':
                Type = 6
            elif len(line) <= 7 and line != '晏几道' and '：' not in line:
                Type = find(dynasty,line)[1]
            try:
                assert(Type== 0 or Type)
            except:
                print(line)
                line = f.readline().replace('\n' , "")
                print(line)
                Type = int(input())
                print(dynasty_tmp[Type])
            if len(line) > 7 or line == '晏几道':
                if "：" not in line:
                    print(line)
                    content = k.split('：')
                if dynasty_tmp[Type] not  in all:
                    all[dynasty_tmp[Type]] = []
                all[dynasty_tmp[Type]].append(line)
            line = f.readline().replace('\n' , "")
all
            

{'汉朝': ['丁兰：“丁兰刻木事亲”为二十四孝故事之一。'],
 '宋朝': ['丁大全：1191-1263 南宋镇江（今属江苏省）官至右司谏、签书枢密院事、右丞相。',
  '丁谓：966-1037 苏州长洲（今江苏省苏州） 北宋真宗时任右谏议大夫、权三司使，后升至宰相，被封为晋国公。其修复京城的故事被尊为古代“运筹学”运用的典范。',
  '丁度：990-1053 祥符（今河南开封） 北宋文字训诂学家，官至端明殿学士。曾与李淑等刊修《韵略》，又刊修《广韵》成《集韵》。',
  '丁顗：祖籍恩州清河（今属河北省） 后迁居祥符，北宋著名藏书家，共搜集图书至八千卷。',
  '丁链：番禺（今属广东省） 北宋元丰年间进士，任朝议郎一职。'],
 '元朝': ['丁鹤年：元惠宗元统三年-明太宗永乐二十二年 武昌 元末明初诗人、养生家，京城老字号“鹤年堂”创始人。',
  '丁德兴：1327—1366 元末定远（今属安徽） 传其祖由拜占庭使华。'],
 '明朝': ['丁士美：1521～1577 淮安府清河县人 明世宗嘉靖三十八年己未科（1559）状元。',
  '丁云鹏：1547～1628 休宁（今属安徽省）人 明代画家，擅画人物，佛像、山水、花卉，亦会作诗。',
  '丁启睿：1595年－1647年 明代河南永城人 诸城丁氏裔孙，官至兵部尚书。',
  '丁魁楚：？—1647年5月8日 明代河南永城人 诸城丁氏裔孙，官至兵部尚书，封平粤伯。',
  '丁耀亢：1599-1669 山东省诸城人 明清文学家。著有小说《续金瓶梅》等'],
 '清朝': ['丁敬：1695—1765 浙江省钱塘（今杭州）人 清代篆刻家，喜好金石文字，善鉴别，工于诗、书、画，尤精刻印，开创了“浙派”，被誉为“西泠八家”之首。',
  '丁丙：1832—1899 浙江省钱塘人 清末藏书家，撰有《善本室藏书志》，著录其藏书中珍贵部分。',
  '丁谦：1843—1919 清末浙江仁和（今杭州）人 字益甫。同治举人。光绪间大挑二等，署汤溪县教谕，旋选授象山县教谕，在任三十年，以造士有绩，升处州府教授，以老不赴。中法战争时，因倡办团练，加强海防有功，奏加五品衔。',
  '丁颖：1888—1964 广东高州人 广东省高州人，现代农学家、水稻专家。曾任中山大学农学院教授。解放后历任华南农学院院长、中国

In [76]:
dynasty = {
"夏朝":[-2070,-1600],           0
"商（殷）朝":[-1600,-1046],     1
"周朝":[-1046,-256],            2
"春秋战国" : [-770,-221],       3
"秦朝":[-221,-206],             4
"汉朝":[-202,263],              5
"三国时期":[220,280],               6
"晋朝":[265,420],               7
"南北朝":[386,589],             8
"隋朝":[581,618],               9
"唐朝":[618,907],               10
"五代十国":[907,960],               11
"宋朝":[960,1279],              12
"元朝":[1271,1368],             13
"明朝":[1368,1644],             14
"清朝":[1636,1912],             15
"中华民国" : [1912,None],        16
"中华人民共和国":[1945,None]      17
}
}

'abc'

In [10]:
#from firstname.models import Famous
flag = False
for i in all:
    for k in all[i]:
        content = k.split('：')
        if  flag:
            print(k,i)
            tmp = Famous(name = content[0] , info = content[1])
            tmp.save()
            try:
                tmp.姓氏.add(content[0][0])
            except:
                try:
                    tmp.姓氏.add(content[0][1])
                except:
                    tmp.姓氏.add(input())
            tmp.朝代.add(i)
            tmp.save()
        elif content[0] == '丘鹤俦' and   content[1] == "1880~1942 广东新宁人 广东音乐演奏家、作曲家、教育家，广东音乐奠基人之一":
            flag = True

丁兰：“丁兰刻木事亲”为二十四孝故事之一。 汉朝
万修：?-26年 陕西兴平此后随刘秀击破邯郸、平定河北。历任偏将军、右将军，先后被封为造义侯、槐里侯。公元26年(建武二年)，万修奉命与扬化将军坚镡共攻南阳郡，因病在军中去世。 汉朝
严忌：约前188-前105 本姓庄，东汉时因避明帝刘庄讳，改为严，后世遂称严忌。严忌少时与司马相如等俱好辞赋，以文才和善辩闻名于时。 汉朝
严助：？—前122年 苏省苏州市 严忌之子，也有人说他是严忌的族子。严助是著名辞赋家。他在汉武帝时任中大夫，其后任会稽太守，在太守任上，并未有出色政绩。建元三年，闽越兵围东瓯，东瓯向汉朝告急，太尉田蚡力主不救，严助和他辩论并取得上风，汉武帝最终出兵援救。严助与朱买臣、淮南王刘安交好，而刘安谋反，严助受御史张汤指控，牵连而诛。 汉朝
严彭祖：西汉东海下邳 严彭祖与颜安乐同学于眭孟习《春秋公羊传》，其后有《春秋公羊传》严氏学及颜氏学。宣帝时，他被立为博士。其后任河南郡太守、太子太傅等职，为人廉直不事权贵。 汉朝
严君平：公元前86年—公元10年 蜀郡成都市 西汉晚期道家学者，思想家。名遵，字君平（东汉班固著《汉书》，因避汉明帝刘庄讳，改写为严君平），蜀郡成都市人。好黄老，汉成帝（前32～前7年在位）时隐居成都市井中，以卜筮为业（以占卜耆龟给人看相），“因势导之以善”，宣扬老子道德经，以惠众人。归隐后著述、设馆授徒于郫县平乐山宣讲《老子》、《庄子》，并在此山上写出了“王莽服诛，光武中兴”的预言，提前20多年预测了“王莽篡权”和“光武中兴”两个重要的历史事件，培养出了得意弟子扬雄，依老子哲学思想，严周著书十余万言写出了一生最重要的几部黄老著作—— 著有《老子注》二卷、《老子指归》十四卷（注与指归本为一书被后人拆分）、《易经骨髓》，使李耳（老子）的道家学说，更加系统条理化，得以发扬光大。《指归》的道论与哲学思想为扬雄、王弼、成玄英等人所继承,成为魏晋玄学所提出的“贵无”、“自然为本”的本体论与重玄学的萌芽。91岁去逝后也埋葬于郫县平乐山。明清时期因疑古派质疑前书已佚，后者今仅存7卷。 汉朝
严延年：东海下邳 西汉酷吏。少习法律，初任郡吏。后历任御史属官、侍御史、御史掾、好畤县令、长史、涿郡太守等职。严延年执法严峻、苛刻、残暴，后因遭人弹劾举报，以诽谤朝政之罪，处于弃市刑罚。 汉朝
严光：前39年—41年 浙江

In [27]:
data = []
for i in all:
    tmp = {}
    tmp['name'] = i
    tmp['surname'] = '丁'
    tmp['symbolSize'] = len(all[i])
    tmp['children'] = []
    for k in all[i]:
        children = {}
        c = k.split('：')
        children['name'] = c[0]
        children['symbolSize'] = 0.1
        children['info'] = c[1]
        tmp['children'].append(children)
    data.append(tmp)
data

[{'name': '汉朝',
  'surname': '丁',
  'value': 1,
  'children': [{'name': '丁兰', 'value': 0.1, 'info': '“丁兰刻木事亲”为二十四孝故事之一。'}]},
 {'name': '宋朝',
  'surname': '丁',
  'value': 5,
  'children': [{'name': '丁大全',
    'value': 0.1,
    'info': '1191-1263 南宋镇江（今属江苏省）官至右司谏、签书枢密院事、右丞相。'},
   {'name': '丁谓',
    'value': 0.1,
    'info': '966-1037 苏州长洲（今江苏省苏州） 北宋真宗时任右谏议大夫、权三司使，后升至宰相，被封为晋国公。其修复京城的故事被尊为古代“运筹学”运用的典范。'},
   {'name': '丁度',
    'value': 0.1,
    'info': '990-1053 祥符（今河南开封） 北宋文字训诂学家，官至端明殿学士。曾与李淑等刊修《韵略》，又刊修《广韵》成《集韵》。'},
   {'name': '丁顗',
    'value': 0.1,
    'info': '祖籍恩州清河（今属河北省） 后迁居祥符，北宋著名藏书家，共搜集图书至八千卷。'},
   {'name': '丁链', 'value': 0.1, 'info': '番禺（今属广东省） 北宋元丰年间进士，任朝议郎一职。'}]},
 {'name': '元朝',
  'surname': '丁',
  'value': 2,
  'children': [{'name': '丁鹤年',
    'value': 0.1,
    'info': '元惠宗元统三年-明太宗永乐二十二年 武昌 元末明初诗人、养生家，京城老字号“鹤年堂”创始人。'},
   {'name': '丁德兴', 'value': 0.1, 'info': '1327—1366 元末定远（今属安徽） 传其祖由拜占庭使华。'}]},
 {'name': '明朝',
  'surname': '丁',
  'value': 5,
  'children': [{'name': '丁士美'

In [46]:
from pyecharts import options as opts
from pyecharts.charts import TreeMap
from pyecharts.commons.utils import JsCode

c = (
    TreeMap()
    .add(
        series_name="option",
        data=data,
        visual_min=300,
        leaf_depth=1,
        # 标签居中为 position = "inside"
        label_opts=opts.LabelOpts(position="inside"),
    )
    .set_global_opts(
        legend_opts=opts.LegendOpts(is_show=False),
        title_opts=opts.TitleOpts(
            title="Echarts 配置项查询分布", subtitle="2016/04", pos_left="leafDepth"
        ),
        tooltip_opts= opts.TooltipOpts(trigger_on='click' ,enterable= True,position = 'inside', hide_delay=1000 , formatter = JsCode(
           """function(params){
               var str = '';
               if(params.data.symbolSize != 0.1)
               {
                   str = params.data.name + ':<br>共有'+params.data.surname + '姓名人' + params.data.symbolSize + '位'
               }
               else
               {
                    str = params.name + ':' + params.data.info.slice(0,20) + '...<br><button><a href="http://www.w3school.com.cn">W3School</a><button>'
               }
                return str
                }
                """
            ) ),
    )
)
c.render()

TypeError: __init__() got an unexpected keyword argument 'enterable'

In [2]:
all = []
path = "F:/数据库/姓氏/"
for i in os.listdir(path):
    data = ""
    tmp_path = path + i + '/'
    if "历代名人.txt" in os.listdir(tmp_path):
        tmp_path +="历代名人.txt"
    else:
        continue
    with open(tmp_path , 'r' , encoding='utf-8') as f:
        line = f.readline().replace('\n','')
        while line:
            if len(line) > 7 or line == '晏几道':
                if "：" in line:
                    content = line.split('：')
                    all.append(content[0])
            line = f.readline().replace('\n' , "")
all

['丁兰',
 '丁大全',
 '丁谓',
 '丁度',
 '丁顗',
 '丁链',
 '丁鹤年',
 '丁德兴',
 '丁士美',
 '丁云鹏',
 '丁启睿',
 '丁魁楚',
 '丁耀亢',
 '丁敬',
 '丁丙',
 '丁谦',
 '丁颖',
 '丁观鹏',
 '丁善庆',
 '丁汝昌',
 '丁惟汾',
 '丁武选',
 '丁玲',
 '丁先国',
 '丁观海',
 '丁本淳',
 '丁世方',
 '丁秋生',
 '丁盛',
 '丁聪',
 '丁甘如',
 '丁一平',
 '丁俊晖',
 '万章',
 '万修',
 '万安国',
 '万宝常',
 '万胜',
 '万敬儒',
 '陶成道',
 '万全',
 '万寿祺',
 '万树',
 '万泰',
 '万经',
 '万斯大',
 '万斯同',
 '万家宝',
 '万籁鸣',
 '万毅',
 '万海峰',
 '万九如',
 '万小元',
 '又尚珍',
 '东良会',
 '东郊',
 '东明珠',
 '东兴明',
 '东国福',
 '东庆臣',
 '东知布',
 '东磊',
 '严君疾',
 '严遂',
 '严忌',
 '严助',
 '严彭祖',
 '严君平',
 '严延年',
 '严光',
 '严颜',
 '严不识',
 '严畯',
 '严棱',
 '严植之',
 '严仁',
 '严蕊',
 '严善思',
 '严震',
 '严武',
 '严绶',
 '严维',
 '严挺之',
 '严浮调',
 '严实',
 '严震直',
 '严嵩',
 '严徵',
 '严我斯',
 '严信厚',
 '严复',
 '严寅亮',
 '严范孙',
 '严文井',
 '乌枝鸣',
 '乌获',
 '乌氏倮',
 '乌获',
 '乌重胤',
 '乌承玼',
 '乌从善',
 '乌竹芳',
 '乌以风',
 '乌·满达夫男',
 '乌·满达夫',
 '乌云',
 '乐祁',
 '乐喜',
 '乐羊',
 '乐毅',
 '乐进',
 '乐广',
 '乐史',
 '乐恢',
 '乐显扬',
 '乐钧',
 '乐松生',
 '乐以琴',
 '乐时鸣',
 '乐茂盛',
 '乐靖宜',
 '大乔',
 '二乔',
 '乔致庸',
 '乔执中',
 '乔知之',
 '乔林',
 '乔维岳',
 '乔吉',
 '乔允升',
 '乔宇',
 '乔

In [3]:
import numpy as np
def transmition(x):
    now = [0,0]
    for k in x:
        if k not in front:
            front[k] = len(front) + 1
            back[len(back) + 1] = k
        now.append(front[k])
    return now
front = {}
back = {}
all = list(map(transmition , all))
all

[[0, 0, 1, 2],
 [0, 0, 1, 3, 4],
 [0, 0, 1, 5],
 [0, 0, 1, 6],
 [0, 0, 1, 7],
 [0, 0, 1, 8],
 [0, 0, 1, 9, 10],
 [0, 0, 1, 11, 12],
 [0, 0, 1, 13, 14],
 [0, 0, 1, 15, 16],
 [0, 0, 1, 17, 18],
 [0, 0, 1, 19, 20],
 [0, 0, 1, 21, 22],
 [0, 0, 1, 23],
 [0, 0, 1, 24],
 [0, 0, 1, 25],
 [0, 0, 1, 26],
 [0, 0, 1, 27, 16],
 [0, 0, 1, 28, 29],
 [0, 0, 1, 30, 31],
 [0, 0, 1, 32, 33],
 [0, 0, 1, 34, 35],
 [0, 0, 1, 36],
 [0, 0, 1, 37, 38],
 [0, 0, 1, 27, 39],
 [0, 0, 1, 40, 41],
 [0, 0, 1, 42, 43],
 [0, 0, 1, 44, 45],
 [0, 0, 1, 46],
 [0, 0, 1, 47],
 [0, 0, 1, 48, 49],
 [0, 0, 1, 50, 51],
 [0, 0, 1, 52, 53],
 [0, 0, 54, 55],
 [0, 0, 54, 56],
 [0, 0, 54, 57, 38],
 [0, 0, 54, 58, 59],
 [0, 0, 54, 60],
 [0, 0, 54, 23, 61],
 [0, 0, 62, 63, 64],
 [0, 0, 54, 4],
 [0, 0, 54, 65, 66],
 [0, 0, 54, 67],
 [0, 0, 54, 68],
 [0, 0, 54, 69],
 [0, 0, 54, 70, 3],
 [0, 0, 54, 70, 71],
 [0, 0, 54, 72, 58],
 [0, 0, 54, 73, 74],
 [0, 0, 54, 75],
 [0, 0, 54, 39, 76],
 [0, 0, 54, 77, 49],
 [0, 0, 54, 78, 79],
 [0, 0, 80

In [4]:
maxlen = 2+4
kind = len(front)
data = tf.keras.preprocessing.sequence.pad_sequences(all ,padding = 'post',truncating='post', maxlen = maxlen)
data

array([[   0,    0,    1,    2,    0,    0],
       [   0,    0,    1,    3,    4,    0],
       [   0,    0,    1,    5,    0,    0],
       ...,
       [   0,    0, 1636,  283,  186,    0],
       [   0,    0, 1636,   34,   45,    0],
       [   0,    0, 1636,  132, 1269,    0]])

In [5]:
def split(data):
    x = []
    y = []
    for i in data:
        for k in range(3):
            x.append(i[k:k+3])
            y.append(i[k+3])
    return np.array(x) , np.array(y)
train_x , train_y = split(data)
train_x , train_y

(array([[   0,    0,    1],
        [   0,    1,    2],
        [   1,    2,    0],
        ...,
        [   0,    0, 1636],
        [   0, 1636,  132],
        [1636,  132, 1269]]),
 array([   2,    0,    0, ...,  132, 1269,    0]))

In [11]:

# model = Sequential([
#     keras.layers.Input((3)),
#     keras.layers.Embedding(3,256),
#     keras.layers.GRU(512 , dropout=0.3 , return_sequences=True),
#     keras.layers.GRU(256 , dropout=0.2 , return_sequences=True),
#     keras.layers.GRU(128 , dropout=0.1),
#     keras.layers.Dense(kind ,activation='softmax')
# ])
x = None
y = None
with open('x' ,'rb') as f:
    x = pickle.load(f)
with open('y' ,'rb') as f:
    y = pickle.load(f)
model = Sequential([
    keras.layers.Input((3)),
    keras.layers.Masking(),
    keras.layers.Embedding(3,512),
    keras.layers.GRU(256 , dropout= 0.1 , return_sequences=True),
    keras.layers.GRU(128 , dropout= 0.1 , return_sequences=True),
    keras.layers.GRU(64 , dropout= 0.1 , return_sequences=True),
    keras.layers.GRU(32 , dropout= 0.1 ),
    keras.layers.Dense(kind , 'softmax' )
])
model.build()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_2 (Masking)          (None, 3)                 0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 3, 512)            1536      
_________________________________________________________________
gru_6 (GRU)                  (None, 3, 256)            591360    
_________________________________________________________________
gru_7 (GRU)                  (None, 3, 128)            148224    
_________________________________________________________________
gru_8 (GRU)                  (None, 3, 64)             37248     
_________________________________________________________________
gru_9 (GRU)                  (None, 32)                9408      
_________________________________________________________________
dense_2 (Dense)              (None, 1922)             

In [12]:
model.compile(loss ='sparse_categorical_crossentropy' ,
 optimizer = keras.optimizers.Adam(0.0002),metrics='acc')

In [14]:
model.fit(x = train_x , y = train_y , batch_size = 256 , epochs=10,validation_split=0.1)

Epoch 1/10
65/65 [==============================] - 1s 22ms/step - loss: 4.4111 - acc: 0.4617 - val_loss: nan - val_acc: 0.4555
Epoch 2/10
65/65 [==============================] - 1s 20ms/step - loss: 4.3736 - acc: 0.4617 - val_loss: nan - val_acc: 0.4555
Epoch 3/10
65/65 [==============================] - 1s 20ms/step - loss: 4.3524 - acc: 0.4617 - val_loss: nan - val_acc: 0.4555
Epoch 4/10
65/65 [==============================] - 1s 19ms/step - loss: 4.3404 - acc: 0.4617 - val_loss: nan - val_acc: 0.4555
Epoch 5/10
65/65 [==============================] - 1s 21ms/step - loss: 4.3336 - acc: 0.4617 - val_loss: nan - val_acc: 0.4555
Epoch 6/10
65/65 [==============================] - 1s 20ms/step - loss: 4.3294 - acc: 0.4617 - val_loss: nan - val_acc: 0.4555
Epoch 7/10
65/65 [==============================] - 1s 21ms/step - loss: 4.3266 - acc: 0.4617 - val_loss: nan - val_acc: 0.4555
Epoch 8/10
65/65 [==============================] - 1s 20ms/step - loss: 4.3246 - acc: 0.4617 - val_loss

In [16]:
model.save_weights('weight')

In [68]:
with open('x', 'wb' ) as f:
    pickle.dump(train_x , f)
with open('y', 'wb' ) as f:
    pickle.dump(train_y , f)

In [13]:
from pyecharts import options as opts
from pyecharts.charts import Tree
data = [{'name': '张', 'symbolSize': 4, 'children': [{'name': '万', 'symbolSize': 1}, {'name': '乌', 'symbolSize': 1}, {'name': '习', 'symbolSize': 1}, {'name': '仇', 'symbolSize': 1}, {'name': '伊', 'symbolSize': 1}, {'name': '伍', 'symbolSize': 1}, {'name': '倪', 'symbolSize': 1}, {'name': '全', 'symbolSize': 1}, {'name': '凤', 'symbolSize': 1}, {'name': '刁', 'symbolSize': 1}, {'name': '利', 'symbolSize': 1}, {'name': '别', 'symbolSize': 1}, {'name': '华', 'symbolSize': 1}, {'name': '卓', 'symbolSize': 1}, {'name': '卫', 'symbolSize': 1}, {'name': '吉', 'symbolSize': 1}, {'name': '吕', 'symbolSize': 1}, {'name': '和', 'symbolSize': 1}, {'name': '奚', 'symbolSize': 1}, {'name': '姚', 'symbolSize': 1}, {'name': '孔', 'symbolSize': 1}, {'name': '安', 'symbolSize': 1}, {'name': '宦', 'symbolSize': 1}, {'name': '寿', 'symbolSize': 1}, {'name': '尹', 'symbolSize': 1}, {'name': '屈', 'symbolSize': 1}, {'name': '岑', 'symbolSize': 1}, {'name': '崔', 'symbolSize': 1}, {'name': '师', 'symbolSize': 1}, {'name': '常', 'symbolSize': 1}, {'name': '平', 'symbolSize': 1}, {'name': '广', 'symbolSize': 1}, {'name': '庄', 'symbolSize': 1}, {'name': '康', 'symbolSize': 1}, {'name': '廖', 'symbolSize': 1}, {'name': '弓', 'symbolSize': 1}, {'name': '张', 'symbolSize': 1}, {'name': '强', 'symbolSize': 1}, {'name': '惠', 'symbolSize': 1}, {'name': '房', 'symbolSize': 1}, {'name': '於', 'symbolSize': 1}, {'name': '易', 'symbolSize': 1}, {'name': '曹', 'symbolSize': 1}, {'name': '朱', 'symbolSize': 1}, {'name': '李', 'symbolSize': 1}, {'name': '杨', 'symbolSize': 1}, {'name': '杭', 'symbolSize': 1}, {'name': '柴', 'symbolSize': 1}, {'name': '桂', 'symbolSize': 1}, {'name': '桑', 'symbolSize': 1}, {'name': '段', 'symbolSize': 1}, {'name': '殷', 'symbolSize': 1}, {'name': '毕', 'symbolSize': 1}, {'name': '水', 'symbolSize': 1}, {'name': '池', 'symbolSize': 1}, {'name': '汪', 'symbolSize': 1}, {'name': '沃', 'symbolSize': 1}, {'name': '湛', 'symbolSize': 1}, {'name': '滑', 'symbolSize': 1}, {'name': '濮', 'symbolSize': 1}, {'name': '燕', 'symbolSize': 1}, {'name': '牛', 'symbolSize': 1}, {'name': '甄', 'symbolSize': 1}, {'name': '白', 'symbolSize': 1}, {'name': '石', 'symbolSize': 1}, {'name': '秦', 'symbolSize': 1}, {'name': '穆', 'symbolSize': 1}, {'name': '窦', 'symbolSize': 1}, {'name': '章', 'symbolSize': 1}, {'name': '童', 'symbolSize': 1}, {'name': '经', 'symbolSize': 1}, {'name': '罗', 'symbolSize': 1}, {'name': '羊', 'symbolSize': 1}, {'name': '翁', 'symbolSize': 1}, {'name': '翟', 'symbolSize': 1}, {'name': '芮', 'symbolSize': 1}, {'name': '范', 'symbolSize': 1}, {'name': '荣', 'symbolSize': 1}, {'name': '蒙', 'symbolSize': 1}, {'name': '蓝', 'symbolSize': 1}, {'name': '蓬', 'symbolSize': 1}, {'name': '计', 'symbolSize': 1}, {'name': '谢', 'symbolSize': 1}, {'name': '谭', 'symbolSize': 1}, {'name': '贝', 'symbolSize': 1}, {'name': '费', 'symbolSize': 1}, {'name': '贾', 'symbolSize': 1}, {'name': '车', 'symbolSize': 1}, {'name': '郏', 'symbolSize': 1}, {'name': '郑', 'symbolSize': 1}, {'name': '金', 'symbolSize': 1}, {'name': '钟', 'symbolSize': 1}, {'name': '闻', 'symbolSize': 1}, {'name': '阎', 'symbolSize': 1}, {'name': '阙', 'symbolSize': 1}, {'name': '隆', 'symbolSize': 1}, {'name': '雷', 'symbolSize': 1}, {'name': '霍', 'symbolSize': 1}, {'name': '韦', 'symbolSize': 1}, {'name': '韩', 'symbolSize': 1}, {'name': '项', 'symbolSize': 1}, {'name': '顾', 'symbolSize': 1}, {'name': '颜', 'symbolSize': 1}, {'name': '鲁', 'symbolSize': 1}, {'name': '鲍', 'symbolSize': 1}, {'name': '齐', 'symbolSize': 1}]}, {'name': '东', 'symbolSize': 1, 'children': [{'name': '张', 'symbolSize': 4, 'children': [{'name': '万', 'symbolSize': 1}, {'name': '乌', 'symbolSize': 1}, {'name': '习', 'symbolSize': 1}, {'name': '仇', 'symbolSize': 1}, {'name': '伊', 'symbolSize': 1}, {'name': '伍', 'symbolSize': 1}, {'name': '倪', 'symbolSize': 1}, {'name': '全', 'symbolSize': 1}, {'name': '凤', 'symbolSize': 1}, {'name': '刁', 'symbolSize': 1}, {'name': '利', 'symbolSize': 1}, {'name': '别', 'symbolSize': 1}, {'name': '华', 'symbolSize': 1}, {'name': '卓', 'symbolSize': 1}, {'name': '卫', 'symbolSize': 1}, {'name': '吉', 'symbolSize': 1}, {'name': '吕', 'symbolSize': 1}, {'name': '和', 'symbolSize': 1}, {'name': '奚', 'symbolSize': 1}, {'name': '姚', 'symbolSize': 1}, {'name': '孔', 'symbolSize': 1}, {'name': '安', 'symbolSize': 1}, {'name': '宦', 'symbolSize': 1}, {'name': '寿', 'symbolSize': 1}, {'name': '尹', 'symbolSize': 1}, {'name': '屈', 'symbolSize': 1}, {'name': '岑', 'symbolSize': 1}, {'name': '崔', 'symbolSize': 1}, {'name': '师', 'symbolSize': 1}, {'name': '常', 'symbolSize': 1}, {'name': '平', 'symbolSize': 1}, {'name': '广', 'symbolSize': 1}, {'name': '庄', 'symbolSize': 1}, {'name': '康', 'symbolSize': 1}, {'name': '廖', 'symbolSize': 1}, {'name': '弓', 'symbolSize': 1}, {'name': '张', 'symbolSize': 1}, {'name': '强', 'symbolSize': 1}, {'name': '惠', 'symbolSize': 1}, {'name': '房', 'symbolSize': 1}, {'name': '於', 'symbolSize': 1}, {'name': '易', 'symbolSize': 1}, {'name': '曹', 'symbolSize': 1}, {'name': '朱', 'symbolSize': 1}, {'name': '李', 'symbolSize': 1}, {'name': '杨', 'symbolSize': 1}, {'name': '杭', 'symbolSize': 1}, {'name': '柴', 'symbolSize': 1}, {'name': '桂', 'symbolSize': 1}, {'name': '桑', 'symbolSize': 1}, {'name': '段', 'symbolSize': 1}, {'name': '殷', 'symbolSize': 1}, {'name': '毕', 'symbolSize': 1}, {'name': '水', 'symbolSize': 1}, {'name': '池', 'symbolSize': 1}, {'name': '汪', 'symbolSize': 1}, {'name': '沃', 'symbolSize': 1}, {'name': '湛', 'symbolSize': 1}, {'name': '滑', 'symbolSize': 1}, {'name': '濮', 'symbolSize': 1}, {'name': '燕', 'symbolSize': 1}, {'name': '牛', 'symbolSize': 1}, {'name': '甄', 'symbolSize': 1}, {'name': '白', 'symbolSize': 1}, {'name': '石', 'symbolSize': 1}, {'name': '秦', 'symbolSize': 1}, {'name': '穆', 'symbolSize': 1}, {'name': '窦', 'symbolSize': 1}, {'name': '章', 'symbolSize': 1}, {'name': '童', 'symbolSize': 1}, {'name': '经', 'symbolSize': 1}, {'name': '罗', 'symbolSize': 1}, {'name': '羊', 'symbolSize': 1}, {'name': '翁', 'symbolSize': 1}, {'name': '翟', 'symbolSize': 1}, {'name': '芮', 'symbolSize': 1}, {'name': '范', 'symbolSize': 1}, {'name': '荣', 'symbolSize': 1}, {'name': '蒙', 'symbolSize': 1}, {'name': '蓝', 'symbolSize': 1}, {'name': '蓬', 'symbolSize': 1}, {'name': '计', 'symbolSize': 1}, {'name': '谢', 'symbolSize': 1}, {'name': '谭', 'symbolSize': 1}, {'name': '贝', 'symbolSize': 1}, {'name': '费', 'symbolSize': 1}, {'name': '贾', 'symbolSize': 1}, {'name': '车', 'symbolSize': 1}, {'name': '郏', 'symbolSize': 1}, {'name': '郑', 'symbolSize': 1}, {'name': '金', 'symbolSize': 1}, {'name': '钟', 'symbolSize': 1}, {'name': '闻', 'symbolSize': 1}, {'name': '阎', 'symbolSize': 1}, {'name': '阙', 'symbolSize': 1}, {'name': '隆', 'symbolSize': 1}, {'name': '雷', 'symbolSize': 1}, {'name': '霍', 'symbolSize': 1}, {'name': '韦', 'symbolSize': 1}, {'name': '韩', 'symbolSize': 1}, {'name': '项', 'symbolSize': 1}, {'name': '顾', 'symbolSize': 1}, {'name': '颜', 'symbolSize': 1}, {'name': '鲁', 'symbolSize': 1}, {'name': '鲍', 'symbolSize': 1}, {'name': '齐', 'symbolSize': 1}]}]}]

In [14]:
c = (
    Tree()
    .add("", data)
    .set_global_opts(title_opts=opts.TitleOpts(title="Tree-基本示例"))
)
c.render_notebook()

In [70]:
links =[{'source': '东', 'target': '张'}, {'source': '乌', 'target': '张'}, {'source': '于', 'target': '张'}, {'source': '云', 'target': '张'}, {'source': '侯', 'target': '张'}, {'source': '古', 'target': '张'}, {'source': '国', 'target': '张'}, {'source': '奚', 'target': '张'}, {'source': '姬', 'target': '张'}, {'source': '孙', 'target': '张'}, {'source': '家', 'target': '张'}, {'source': '封', 'target': '张'}, {'source': '尚', 'target': '张'}, {'source': '居', 'target': '张'}, {'source': '山', 'target': '张'}, {'source': '弓', 'target': '张'}, {'source': '张', 'target': '张'}, {'source': '成', 'target': '张'}, {'source': '支', 'target': '张'}, {'source': '时', 'target': '张'}, {'source': '昌', 'target': '张'}, {'source': '明', 'target': '张'}, {'source': '满', 'target': '张'}, {'source': '濮', 'target': '张'}, {'source': '禄', 'target': '张'}, {'source': '秋', 'target': '张'}, {'source': '章', 'target': '张'}, {'source': '范', 'target': '张'}, {'source': '葛', 'target': '张'}, {'source': '解', 'target': '张'}, {'source': '诸', 'target': '张'}, {'source': '赵', 'target': '张'}, {'source': '都', 'target': '张'}, {'source': '高', 'target': '张'}, {'source': '魏', 'target': '张'}, {'source': '黄', 'target': '张'}, {'source': '黎', 'target': '张'}]

In [84]:
nodes =[ {'name': '张', 'symbolSize': 20},{'name': '东', 'symbolSize': 10}, {'name': '乌', 'symbolSize': 10}, {'name': '于', 'symbolSize': 10}, {'name': '云', 'symbolSize': 10}, {'name': '侯', 'symbolSize': 10}, {'name': '古', 'symbolSize': 10}, {'name': '国', 'symbolSize': 10}, {'name': '奚', 'symbolSize': 10}, {'name': '姬', 'symbolSize': 10}, {'name': '孙', 'symbolSize': 10}, {'name': '家', 'symbolSize': 10}, {'name': '封', 'symbolSize': 10}, {'name': '尚', 'symbolSize': 10}, {'name': '居', 'symbolSize': 10}, {'name': '山', 'symbolSize': 10}, {'name': '弓', 'symbolSize': 10}, {'name': '成', 'symbolSize': 10}, {'name': '支', 'symbolSize': 10}, {'name': '时', 'symbolSize': 10}, {'name': '昌', 'symbolSize': 10}, {'name': '明', 'symbolSize': 10}, {'name': '满', 'symbolSize': 10}, {'name': '濮', 'symbolSize': 10}, {'name': '禄', 'symbolSize': 10}, {'name': '秋', 'symbolSize': 10}, {'name': '章', 'symbolSize': 10}, {'name': '范', 'symbolSize': 10}, {'name': '葛', 'symbolSize': 10}, {'name': '解', 'symbolSize': 10}, {'name': '诸', 'symbolSize': 10}, {'name': '赵', 'symbolSize': 10}, {'name': '都', 'symbolSize': 10}, {'name': '高', 'symbolSize': 10}, {'name': '魏', 'symbolSize': 10}, {'name': '黄', 'symbolSize': 10}, {'name': '黎', 'symbolSize': 10}]

In [80]:
from pyecharts import options as opts
from pyecharts.charts import Graph

In [85]:
# nodes = [    {'name': '东', 'symbolSize': 10}, {'name': '乌', 'symbolSize': 10}, {'name': '于', 'symbolSize': 10}, {'name': '云', 'symbolSize': 10}, {'name': '侯', 'symbolSize': 10}, {'name': '解', 'symbolSize': 10}, {'name': '诸', 'symbolSize': 10}, {'name': '赵', 'symbolSize': 10}, {'name': '都', 'symbolSize': 10}, {'name': '高', 'symbolSize': 10}, {'name': '魏', 'symbolSize': 10}, {'name': '黄', 'symbolSize': 10}, {'name': '黎', 'symbolSize': 10} ,{'name': '张', 'symbolSize': 20},{'name': '成', 'symbolSize': 10}, {'name': '支', 'symbolSize': 10}, {'name': '时', 'symbolSize': 10}, {'name': '昌', 'symbolSize': 10}, {'name': '明', 'symbolSize': 10}
# ]
# links = []
# for i in nodes:
#     for j in nodes:
#         links.append({"source": i.get("name"), "target": '张'})
c = (
    Graph()
    .add("", nodes, links, repulsion=80,linestyle_opts=opts.LineStyleOpts(curve=0.2),
        label_opts=opts.LabelOpts(is_show=False),)
    .set_global_opts(title_opts=opts.TitleOpts(title="Graph-基本示例"))
)
c.render_notebook()

In [86]:
def split_sents(para:str):
    sents=[]
    start_ind=0
    for i in range(len(para)):
        w=para[i]
        if w=='。' or w==';' or w=='\r':
            if i>start_ind:
                s=para[start_ind:i]
                if s!='':
                    sents.append(s)
            if w=='\r':
                i+=1
            start_ind=i+1

    if len(para)>start_ind:
        s = para[start_ind:]
        if s != '':
            sents.append(s)

    return sents

def get_dynasty_migration(migration_history,show_details=False):
    dynasties_find='夏,商,周,秦,汉,三国,晋,南朝,北朝,南北朝,隋,唐,五代,十国,宋,金,元,明,清,民国'.split(',')
    dynasties='夏朝,商（殷）朝,周朝,春秋战国,秦朝,汉朝,三国时期,晋朝,南北朝,隋朝,唐朝,五代十国,宋朝,元朝,明朝,清朝,中华民国,中华人民共和国'.split(',')
    provinces='内蒙古,新疆,广西,西藏,重庆,北京,天津,澳门,香港,河北,山西,吉林,辽宁,黑龙江,陕西,甘肃,青海,' \
              '山东,福建,浙江,台湾,河南,湖北,湖南,江西,江苏,安徽,广东,海南,四川,贵州,云南,宁夏,上海'.split(',')

    sents=split_sents(migration_history)
    migration_locations={}
    # dynasty_locations = []
    validation=[]
    time_ind=0
    for i in range(len(sents)):
        s=sents[i]

        find_dynasties=[]
        for dy_ind,d in enumerate(dynasties_find):
            if d in s and dy_ind>=time_ind:
                find_dynasties.append(d)
                time_ind=dy_ind

        if len(find_dynasties) == 0:
            continue

        find_provinces=[]
        for p in provinces:
            if p in s:
                find_provinces.append(p)

        if len(find_provinces)==0:
            continue

        for d in find_dynasties:
           for p in find_provinces:
               for dy in dynasties:
                   if d in dy:
                       loc_set=migration_locations.setdefault(dy, [])
                       loc_set.append(p)

        validation.append([s,find_dynasties,find_provinces])

    # for d in dynasties:
    #     if d in migration_locations:
    #         dynasty_locations.append([d,migration_locations[d]])

    if show_details:
        for item in validation:
            print(item)

    return migration_locations

In [87]:
get_dynasty_migration("""万氏是一个多民族、多源流的古老姓氏，在当今姓氏排行榜上名列第八十八位，属于大姓系列，人口约二百九十五万三千余，占全国人口总数的0.18%左右。
春秋时期，芮国占据今山西、陕西间地，晋国则占山西一带，故早期万氏当发源于山西、陕西省境，此二省亦为后世万氏支系的主要源头。
汉朝以前，万氏族人有入迁山东者，如战国时期有山东人万章，但两发源地仍为万氏族人主要繁衍生息之地。两汉时期，陕西扶风一带万氏族人枝繁叶茂，遂有郡望——扶风郡。
魏、晋、南北朝时，北方战火四起，万氏族人有避居南方者，此次南迁，奠定了后世万氏族人盛于南方的基础。另外，北魏迁都洛阳后，随着汉化政策的推行，鲜卑复姓吐万氏改单姓万，并迅速发展壮大，形成又一郡望——河南郡。
唐朝时期，浙江、安徽万氏较盛，时有浙江昆山令万齐融；安徽人万敬儒，三世同居，族人所居之所称“成孝乡广孝聚”。唐代以后，史载万氏名人比比皆是，且多出南方。
宋、元时期，更朝换代频繁，少数民族辽、金、蒙皆想争霸天下，北方时有战争，致使万氏族人举族南迁，南方万氏族人由此陡增，江西、湖北、湖南、天津、山东等地均有万氏族人散居点。据载，宋朝有万顷，出江州(今江西九江)；万镇，出平江(今湖南平江)；万人杰，出大冶(今湖北大冶)；元朝有万邦宁，出宝坻(今天津)。
明、清时期，是历史上万氏族人最盛阶段，山东、四川、江苏、广东、广西亦有万氏族人的足迹。可见，万氏族人已涉足于祖国最南端，遂有清朝万氏族人渡海迁播于更广阔之领域。据此期间史载万氏名人来看，多出、江苏、江西一带，则此区诚为万氏族人发展最旺之地。另，明朝万氏作为大槐树迁民姓氏之一，于河北、河南、山东、安徽、陕西、北京等地落籍。
如今，万氏族人尤以山东、江西、江苏、湖北等省多此姓，这四省万氏族人约占全国万氏人口的40%以上。""")

{'晋朝': ['山西', '陕西'],
 '唐朝': ['浙江', '安徽'],
 '宋朝': ['天津', '山东', '湖北', '湖南', '江西'],
 '元朝': ['天津', '山东', '湖北', '湖南', '江西', '天津', '湖北', '湖南', '江西'],
 '明朝': ['广西', '山东', '江苏', '广东', '四川'],
 '清朝': ['广西', '山东', '江苏', '广东', '四川']}

In [88]:
import pandas as pd

In [98]:
surname = pd.read_csv('main_surname.csv' , header = None).fillna("")
province = pd.read_csv('main_province.csv' , header= None)
mig = surname[3]
province_surname = []
surname_dynasty = []
for i,j in zip(surname[0] , mig):
    

丁 
万 万氏是一个多民族、多源流的古老姓氏，在当今姓氏排行榜上名列第八十八位，属于大姓系列，人口约二百九十五万三千余，占全国人口总数的0.18%左右。
春秋时期，芮国占据今山西、陕西间地，晋国则占山西一带，故早期万氏当发源于山西、陕西省境，此二省亦为后世万氏支系的主要源头。
汉朝以前，万氏族人有入迁山东者，如战国时期有山东人万章，但两发源地仍为万氏族人主要繁衍生息之地。两汉时期，陕西扶风一带万氏族人枝繁叶茂，遂有郡望——扶风郡。
魏、晋、南北朝时，北方战火四起，万氏族人有避居南方者，此次南迁，奠定了后世万氏族人盛于南方的基础。另外，北魏迁都洛阳后，随着汉化政策的推行，鲜卑复姓吐万氏改单姓万，并迅速发展壮大，形成又一郡望——河南郡。
唐朝时期，浙江、安徽万氏较盛，时有浙江昆山令万齐融；安徽人万敬儒，三世同居，族人所居之所称“成孝乡广孝聚”。唐代以后，史载万氏名人比比皆是，且多出南方。
宋、元时期，更朝换代频繁，少数民族辽、金、蒙皆想争霸天下，北方时有战争，致使万氏族人举族南迁，南方万氏族人由此陡增，江西、湖北、湖南、天津、山东等地均有万氏族人散居点。据载，宋朝有万顷，出江州(今江西九江)；万镇，出平江(今湖南平江)；万人杰，出大冶(今湖北大冶)；元朝有万邦宁，出宝坻(今天津)。
明、清时期，是历史上万氏族人最盛阶段，山东、四川、江苏、广东、广西亦有万氏族人的足迹。可见，万氏族人已涉足于祖国最南端，遂有清朝万氏族人渡海迁播于更广阔之领域。据此期间史载万氏名人来看，多出、江苏、江西一带，则此区诚为万氏族人发展最旺之地。另，明朝万氏作为大槐树迁民姓氏之一，于河北、河南、山东、安徽、陕西、北京等地落籍。
如今，万氏族人尤以山东、江西、江苏、湖北等省多此姓，这四省万氏族人约占全国万氏人口的40%以上。
上官 
东 东氏人口总数在中国的大陆和台湾省均未列入百家姓前三百位，在宋版《百家姓》中排序为第三百六十位门阀。东姓在大陆和台湾都没有列入百家姓前一百位。相传伏羲氏为东方部族的首领，而他的部落的发展最为迅速，生产方式最先进，伏羲的部落在华夏发展史上有不可磨灭的伟绩，他的后代中就以东为姓，借以纪念他在东方部落的贡献。而在先帝舜时，他有七友，其中之一名为东不訾，他的后代都用东为姓。
东氏目前在中国分布较广，今河南省的沈丘县，江苏省的海门市、通州市，河北省的霸州、沧州市青县、邢台市隆尧

In [100]:
from pyecharts import options as opts
from pyecharts.charts import Map, Timeline
from pyecharts.faker import Faker

tl = Timeline()
for i in range(2015, 2020):
    print([list(z) for z in zip(Faker.provinces, Faker.values())])
    map0 = (
        Map()
        .add("商家A", [list(z) for z in zip(Faker.provinces, Faker.values())], "china")
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Map-{}年某些数据".format(i)),
            visualmap_opts=opts.VisualMapOpts(max_=200),
        )
    )
    tl.add(map0, "{}年".format(i))
tl.render("timeline_map.html")

[['广东', 86], ['北京', 88], ['上海', 108], ['江西', 112], ['湖南', 92], ['浙江', 141], ['江苏', 92]]
[['广东', 63], ['北京', 115], ['上海', 30], ['江西', 79], ['湖南', 67], ['浙江', 42], ['江苏', 24]]
[['广东', 119], ['北京', 70], ['上海', 101], ['江西', 130], ['湖南', 121], ['浙江', 72], ['江苏', 76]]
[['广东', 129], ['北京', 121], ['上海', 87], ['江西', 70], ['湖南', 68], ['浙江', 77], ['江苏', 86]]
[['广东', 113], ['北京', 112], ['上海', 133], ['江西', 47], ['湖南', 128], ['浙江', 107], ['江苏', 103]]


'e:\\project\\timeline_map.html'